To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# ⚖️ Contract Risk Tagging

**Hook:** *"AI that reads legal contracts and flags risks instantly"*

Legal review takes weeks. Contract risks hide in dense legalese. Watch Fenic extract structured risk data from contract clauses in seconds - liability caps, termination rights, indemnification issues. This is AI-powered legal analysis that scales.

**What you'll see in this 2-minute demo:**
- 📋 **Contract clauses** - Real liability, termination, and indemnification language
- 🎯 **Risk extraction** - Structured data from unstructured legal text
- ⚠️ **Severity scoring** - Quantified risk levels (1-5 scale)
- 🔍 **Explanation text** - Human-readable risk summaries for legal teams

Perfect for contract management, legal tech, and due diligence workflows.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import Literal

# ⚡ Configure for legal document analysis
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="contract_risk_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "legal": fc.OpenAILanguageModel(model_name="gpt-4o", rpm=100, tpm=100_000),
            # "legal": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "legal": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=100, tpm=100_000)
        }
    )
))

print("✅ Legal analysis session configured with OpenAI GPT-4o")

## 📋 Step 1: Contract Risk Schema

Define exactly what legal risks we want to extract from contract language:

In [ ]:
# ⚖️ Structured legal risk schema
class ContractRisk(BaseModel):
    risk_type: Literal["liability", "termination", "indemnification", "payment"] = Field(description="Type of legal risk identified")
    severity: int = Field(ge=1, le=5, description="Risk level (1=low, 5=critical)")
    description: str = Field(description="Brief explanation of the risk")
    recommendation: str = Field(description="Suggested action or negotiation point")
    affects_party: Literal["us", "them", "mutual"] = Field(description="Which party bears the risk")

print("📋 Legal Risk Extraction Schema:")
print("   • risk_type: Category of legal risk (liability/termination/indemnification/payment)")
print("   • severity: 1-5 criticality score")
print("   • description: Human-readable risk summary")
print("   • recommendation: Legal team guidance")
print("   • affects_party: Risk allocation analysis (us/them/mutual)")

## 📄 Step 2: Real Contract Clauses

Analyze actual contract language with hidden risks and problematic terms:

In [ ]:
# 📄 Real contract clauses with various risk levels
contract_clauses = session.create_dataframe([
    {
        "clause_id": "TERM-001", 
        "clause_type": "Termination",
        "text": "Either party may terminate this agreement with 30 days written notice. Upon termination, Company shall immediately cease all work and return confidential information."
    },
    {
        "clause_id": "LIAB-002", 
        "clause_type": "Liability",
        "text": "Company's total liability under this agreement shall not exceed the total amount paid by Client in the twelve months preceding the claim. Company excludes all liability for consequential damages."
    },
    {
        "clause_id": "INDEM-003", 
        "clause_type": "Indemnification",
        "text": "Company shall defend, indemnify and hold Client harmless against all claims, damages, costs and expenses arising from Company's breach of this agreement or negligent performance of services."
    },
    {
        "clause_id": "PAY-004", 
        "clause_type": "Payment",
        "text": "Payment terms are Net 90 days from invoice date. Client may withhold payment for any disputed amounts until resolution. Late payments incur 1.5% monthly interest."
    }
])

print("📄 Contract Clauses for Risk Analysis:")
contract_clauses.show()

## ⚠️ Step 3: AI Risk Extraction

Extract structured legal risks using AI that understands contract language:

In [ ]:
# ⚖️ Extract legal risks from contract text
risk_analysis = contract_clauses.select(
    "clause_id",
    "clause_type", 
    "text",
    fc.semantic.extract(
        "text",
        ContractRisk,
        model_alias="legal"
    ).alias("risk_analysis")
).cache()  # Cache legal analysis results

# Extract structured fields for analysis
risk_summary = risk_analysis.select(
    "clause_id",
    "clause_type",
    risk_analysis.risk_analysis.risk_type.alias("risk_type"),
    risk_analysis.risk_analysis.severity.alias("severity"),
    risk_analysis.risk_analysis.affects_party.alias("affects_party"),
    risk_analysis.risk_analysis.description.alias("description")
)

print("⚠️ CONTRACT RISK ANALYSIS:")
risk_summary.show()

## 🔍 Step 4: Legal Risk Dashboard

Summarize findings for legal teams with actionable insights:

In [ ]:
# 📊 Legal risk dashboard analysis
high_risk = risk_summary.filter(fc.col("severity") >= 4).count()
total_risks = risk_summary.count()
critical_percentage = (high_risk / total_risks) * 100

print("🔍 LEGAL RISK DASHBOARD:")
print(f"   • Total clauses analyzed: {total_risks}")
print(f"   • High-risk clauses (4-5): {high_risk} ({critical_percentage:.0f}%)")

# Show recommendations for high-risk items
print("\n⚠️ HIGH-RISK ITEMS:")
high_risk_details = risk_analysis.filter(
    risk_analysis.risk_analysis.severity >= 4
).select(
    "clause_type",
    risk_analysis.risk_analysis.recommendation.alias("recommendation")
)
high_risk_details.show()

print("\n💼 BUSINESS IMPACT:")
print("   • Automated legal review reduces time from weeks to minutes")
print("   • Structured risk data enables consistent contract analysis")
print("   • AI flags issues human reviewers might miss")
print("   • Risk scoring helps prioritize negotiation efforts")

In [ ]:
session.stop()